In [18]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import time

In [2]:
from tokenize_data import process

directory = '/home/a_stasiuk/bsuir_ml_part2/data/aclImdb/train'

vectors, labels = process(directory, mode='train', format='bag-word')

  0%|          | 38/12500 [00:00<00:33, 369.93it/s]

/home/a_stasiuk/bsuir_ml_part2/data/aclImdb/train/pos


  0%|          | 38/12500 [00:00<00:33, 372.65it/s]

/home/a_stasiuk/bsuir_ml_part2/data/aclImdb/train/neg


100%|██████████| 12500/12500 [00:31<00:00, 400.32it/s]


In [29]:
batch_size = 1024
train_data = TensorDataset(torch.Tensor(vectors), torch.Tensor(labels))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [30]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [76]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden


In [79]:
def train(train_loader, learn_rate, hidden_dim=512, EPOCHS=10, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[1]
#     print(input_dim)
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            x.unsqueeze_(0)
            x = x.permute(1,0,2)
            
            out, h = model(x.to(device).float(), h)
            
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

In [80]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")

Starting Training of GRU model


/home/a_stasiuk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
/home/a_stasiuk/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Epoch 1/10 Done, Total Loss: 0.39306483790278435
Total Time Elapsed: 0.2482479999999896 seconds
Epoch 2/10 Done, Total Loss: 0.2842579322556655
Total Time Elapsed: 0.24003600000000347 seconds
Epoch 3/10 Done, Total Loss: 0.2561418575545152
Total Time Elapsed: 0.24236000000000502 seconds
Epoch 4/10 Done, Total Loss: 0.2508007735013962
Total Time Elapsed: 0.3215900000000005 seconds
Epoch 5/10 Done, Total Loss: 0.2516236739854018
Total Time Elapsed: 0.2417599999999993 seconds
Epoch 6/10 Done, Total Loss: 0.25101543093721074
Total Time Elapsed: 0.24027199999999027 seconds
Epoch 7/10 Done, Total Loss: 0.2535048232724269
Total Time Elapsed: 0.3247829999999965 seconds
Epoch 8/10 Done, Total Loss: 0.25166815891861916
Total Time Elapsed: 0.24022399999999777 seconds
Epoch 9/10 Done, Total Loss: 0.25966268715759117
Total Time Elapsed: 0.24185799999999347 seconds
Epoch 10/10 Done, Total Loss: 0.25469113886356354
Total Time Elapsed: 0.32337499999999864 seconds
Total Training Time: 2.664505999999974